In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias = False) # 28 > 26 | 3
        self.b1 = nn.BatchNorm2d(8)
        self.d1 = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1, bias = False)  # 26 > 24 | 5
        self.b2 = nn.BatchNorm2d(8)
        self.d2 = nn.Dropout2d(0.1)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1, bias = False) # 24 > 22 | 7
        self.b3 = nn.BatchNorm2d(16)
        self.d3 = nn.Dropout2d(0.1)
        self.pool1 = nn.MaxPool2d(2, 2)                           # 22 > 11


        self.conv4 = nn.Conv2d(16, 16, 3, padding=1, bias = False) # 11 > 9 | 9
        self.b4 = nn.BatchNorm2d(16)
        self.d4 = nn.Dropout2d(0.1)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1, bias = False)  # 9 > 7 | 11
        self.b5 = nn.BatchNorm2d(16)
        self.d5 = nn.Dropout2d(0.1)
        self.conv6 = nn.Conv2d(16, 24, 3, padding=1, bias = False)  # 7 > 5 | 13
        self.b6 = nn.BatchNorm2d(24)
        self.d6 = nn.Dropout2d(0.1)
        self.conv7 = nn.Conv2d(24, 24, 3, padding=1, bias = False) # 5 > 3  | 15
        self.b7 = nn.BatchNorm2d(24)
        self.d7 = nn.Dropout2d(0.1)
        self.conv1c = nn.Conv2d(24, 10, 3, bias = False)           # 3 > 1 | 17
        self.gap = nn.AdaptiveAvgPool2d((1,1))                     # 1 > 1 | 17

    def forward(self, x):
        x = self.pool1(self.d3(self.b3(F.relu(self.conv3(self.d2(self.b2(F.relu(self.conv2(self.d1(self.b1(F.relu(self.conv1(x)))))))))))))
        x = self.d5(self.b5(F.relu(self.conv5(self.d4(self.b4(F.relu(self.conv4(x))))))))
        x = self.d6(self.b6(F.relu(self.conv6(x))))
        x = F.avg_pool2d(x,(4,4))
        x = self.conv1c(self.d7(self.b7(F.relu(self.conv7(x)))))
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             576
       BatchNorm2d-5            [-1, 8, 28, 28]              16
         Dropout2d-6            [-1, 8, 28, 28]               0
            Conv2d-7           [-1, 16, 28, 28]           1,152
       BatchNorm2d-8           [-1, 16, 28, 28]              32
         Dropout2d-9           [-1, 16, 28, 28]               0
        MaxPool2d-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,304
      BatchNorm2d-12           [-1, 16, 14, 14]              32
        Dropout2d-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0410, Accuracy: 9872/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0302, Accuracy: 9906/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0157, Accuracy: 9942/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0135, Accuracy: 9954/10000 (100%)



loss=0.006376375909894705 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.93it/s]



Test set: Average loss: 0.0147, Accuracy: 9950/10000 (100%)

